In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from gensim.parsing.porter import PorterStemmer


# spacy for lemmatization
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
import nltk#; nltk.download('stopwords')
#nltk.data.path.append('C:/Users/Milena/AppData/Roaming/nltk_data')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

### Read in all text

In [ ]:
import glob

path = "C:/Users/mdjou/OneDrive/Desktop/craigslist-replication-code-and-data/data/Newspapers_content/Topic_Model/Sample_dates/clean_text/__*.xlsx"

glob.glob(path)

df = pd.DataFrame()
for f in glob.glob(path):
    df_ = pd.read_excel(f)
    df = df.append(df_,ignore_index=True)


C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ip

C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ipykernel_19336\1718532052.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_,ignore_index=True)
C:\Users\mdjou\AppData\Local\Temp\ip

In [ ]:
df.head()
print(len(df))

In [ ]:
df.text = df.alltext
df.text = df.text.astype(str)

### Pre-process the text: remove punctuation, stop words, stem

In [ ]:
longlist = pd.read_csv('C:/Users/mdjou/OneDrive/Desktop/craigslist-replication-code-and-data/data/Newspapers_content/Topic_Model/__very_long_stopwords.txt', header=None)
addlist  = pd.read_csv('C:/Users/mdjou/OneDrive/Desktop/craigslist-replication-code-and-data/data/Newspapers_content/Topic_Model/__additional_stopwords.txt', header=None)
longlist = longlist[0].tolist()
addlist  =  addlist[0].tolist()

In [ ]:
stop_words = stop_words+longlist+addlist
print(len(stop_words))

In [ ]:
def sent_to_words(sentences):
    output = []
    for sentence in sentences:
        output.append(simple_preprocess(sentence))  # deacc=True removes punctuations
    return output

def remove_stopwords(texts):
    output = []
    for text in texts:
        output.append([w for w in text if w not in stop_words])
    return output


In [ ]:
data = sent_to_words(df.alltext)

data = remove_stopwords(data)

data = [PorterStemmer().stem_documents(text) for text in data]

In [ ]:
data[1:10]

### Construct corpus, filter extremes

In [ ]:
### define corpus

id2word = corpora.Dictionary(data)

print('Number of unique tokens before trim: %d' % len(set(id2word)))

#### Use filter_extremes to keep words encountered in at least 200 documents / no more than 20% of the total sample
id2word.filter_extremes(no_below=200, no_above=0.2)

print('Number of unique tokens after trim: %d' % len(set(id2word)))

corpus = [id2word.doc2bow(text) for text in data]


In [ ]:
corpus_new = [id2word.doc2bow(text) for text in data]

In [ ]:
doc_word = gensim.matutils.corpus2csc(corpus_new)
doc_word = doc_word.transpose()
doc_word.shape

In [ ]:
words = list(id2word.token2id.keys())

# CorEx model: No anchoring

In [ ]:
from corextopic import corextopic as ct

corex_model = ct.Corex(n_hidden=5, words=words, max_iter=200, verbose=False, seed=42)
corex_model.fit(doc_word, words=words)

In [ ]:
# Print topics
topics = corex_model.get_topics()
for n,topic in enumerate(topics):
    topic_words, x, y = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
top_docs = corex_model.get_top_docs()
#for topic_n, topic_docs in enumerate(top_docs):
#    docs,probs = zip(*topic_docs)
#    topic_str = str(topic_n+1)+': '+', '.join(docs)
#   print(topic_str)
print(top_docs[1])

In [ ]:
topics = corex_model.p_y_given_x
print(len(topics[:,4])) # n_docs x k_topics
print(topics)

In [ ]:
df = pd.DataFrame({
 'year'       : df['year'],
 'source'     : df['source'],
 'topic0'     : topics[:,0],
 'topic1'     : topics[:,1],
 'topic2'     : topics[:,2],
 'topic3'     : topics[:,3],
 'topic4'     : topics[:,4]
 #'topic5'     : topics[:,5],
 #'topic6'     : topics[:,6],
 #'topic7'     : topics[:,7],
 #'topic8'     : topics[:,8],
 #'topic9'     : topics[:,9] 
 })


print(df)
print(len(df.year))

In [ ]:
avg_topics = pd.DataFrame(df.groupby(['source', 'year']).agg(['mean'])[['topic0', 'topic1', 'topic2', 'topic3', 'topic4']].reset_index())
avg_topics.to_excel('C:/Users/mdjou/OneDrive/Desktop/craigslist-replication-code-and-data/data/Newspapers_content/Topic_Model/avg_topic_prob_unanchored_by_newspaper_year.xlsx')

In [ ]:
labels = corex_model.labels
print(len(labels[:,9])) # n_docs x k_topics
print(labels[:,0])

## Corex with Anchoring -- Anchors for 4 political topics

In [ ]:

corex_model.fit(doc_word, 
                words=words, 
                anchors=[['washington', 'feder', 'govern', 'presid'], 
                         ['council', 'mayor'],
                         ['repres', 'congress', 'senat'],
                         ['intern', 'abroad', 'foreign']],
                     anchor_strength=2)

In [ ]:
# Print topics

topics = corex_model.get_topics()

for n,topic in enumerate(topics):
    topic_words, x, y = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

### Extract topic probabilities by article

In [ ]:
topics = corex_model.p_y_given_x
print(len(topics[:,9])) # n_docs x k_topics
print(topics)

### Collapse by newspaper and export

In [ ]:
df = pd.DataFrame({
 'year'       : df['year'],
 'source'     : df['source'],
 'topic0'     : topics[:,0],
 'topic1'     : topics[:,1],
 'topic2'     : topics[:,2],
 'topic3'     : topics[:,3],
 'topic4'     : topics[:,4],
 'topic5'     : topics[:,5],
 'topic6'     : topics[:,6],
 'topic7'     : topics[:,7],
 'topic8'     : topics[:,8],
 'topic9'     : topics[:,9] 
 })


print(df)
print(len(df.year))

### Export collapsed dataset: average topic probabilities by newspaper and year

In [ ]:
avg_topics = pd.DataFrame(df.groupby(['source', 'year']).agg(['mean'])[['topic0', 'topic1', 'topic2', 'topic3', 'topic4', 'topic5', 'topic6', 'topic7', 'topic8', 'topic9']].reset_index())
avg_topics.to_excel('C:/Users/mdjou/OneDrive/Desktop/craigslist-replication-code-and-data/data/Newspapers_content/Topic_Model/avg_topic_prob_by_newspaper_year.xlsx') 